In [54]:
import pandas as pd
import numpy as np
import re
import random
import math

In [55]:
# !pip install bs4
# !pip install bert-for-tf2
# !pip install sentencepiece


In [56]:
import tensorflow as tf
import tensorflow_hub as hub
from bs4 import BeautifulSoup
from tensorflow.keras import layers
import bert

In [57]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [58]:
print(tf.__version__)

2.4.1


# 1. DataPreprocessing

In [59]:
cols = ["sentiment","id","date","query","user","text"]
dataset = pd.read_csv("/content/drive/MyDrive/BERT INTUITION/training.1600000.processed.noemoticon.csv",
                      header = None,
                      engine = "python",
                      encoding = "latin1",
                      names = cols)

In [60]:
dataset.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [61]:
dataset.drop(["id","query","user","date"] ,axis = 1 , inplace = True)

In [62]:
dataset.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [63]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  # removing all the mentions
  tweet = re.sub(r'@[A-Za-z0-9]+',' ',tweet)
  # removing all the urls
  tweet = re.sub(r'https?://[A-Za-z0-9./]+',' ',tweet)
  # keeping all the letters
  tweet = re.sub(r"[^A-Za-z0-9.!?']",' ',tweet)
  # removing all the whitespaces
  tweet = re.sub(r' +',' ',tweet)

  return tweet
  

In [64]:
data_clean = [clean_tweet(tweet) for tweet in dataset.text]

In [65]:
data_labels = dataset.sentiment.values
data_labels[data_labels == 4] = 1

In [66]:
data_clean[0]

" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D"

# 2. Tokenization

In [67]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file=vocab_file,do_lower_case=do_lower_case)

In [ ]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
data_inputs = [encode_sentence(sent) for sent in data_clean]

In [68]:
word_data = [tokenizer.tokenize(sent) for sent in data_clean]
word_data[0]

['aw',
 '##w',
 '##w',
 'that',
 "'",
 's',
 'a',
 'bum',
 '##mer',
 '.',
 'you',
 'should',
 '##a',
 'got',
 'david',
 'carr',
 'of',
 'third',
 'day',
 'to',
 'do',
 'it',
 '.',
 'd']

In [ ]:
how

# 3. Dataset Creation

In [19]:
data_with_len = [[sent , data_labels[idx] , len(sent)]
                 for idx , sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x:x[2])
sorted_all = [(sent[0],sent[1]) for sent in data_with_len if sent[2] > 7]

In [20]:
data_with_len[0]

[[], 1, 0]

In [21]:
sorted_all[0]

([3062, 2125, 1996, 2227, 1997, 1996, 3011, 6719], 0)

In [22]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types = (tf.int32,tf.int32))

In [23]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=array([3062, 2125, 1996, 2227, 1997, 1996, 3011, 6719], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [24]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(batch_size = BATCH_SIZE,padded_shapes = ((None,),()))

In [25]:
all_batched

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [26]:
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES//10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [27]:
print(NB_BATCHES)
print(NB_BATCHES_TEST)
print(test_dataset)
print(train_dataset)

41495
4149
<TakeDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>
<SkipDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>


# 4. Model Building

In [28]:
class DCNN(tf.keras.Model):
  def __init__(self,
               vocab_size,
               emb_dim = 128,
               nb_filters = 50,
               FFN_units = 512,
               nb_classes = 2,
               dropout_rate = 0.1,
               training = False,
               name = 'dcnn'):
    super(DCNN,self).__init__(name = name)

    self.embedding = layers.Embedding(vocab_size , emb_dim)
    self.bigram = layers.Conv1D(filters=nb_filters , 
                                kernal_size = 2,
                                padding = 'valid',
                                activation = 'relu')
    self.trigram = layers.Conv1D(filters=nb_filters , 
                                kernal_size = 3,
                                padding = 'valid',
                                activation = 'relu')
    self.fourgram = layers.Conv1D(filters=nb_filters , 
                                kernal_size = 4,
                                padding = 'valid',
                                activation = 'relu')
    self.pool = layers.GlobalMaxPooling1D()
    self.dense_1 = layers.Dense(units = FFN_units , activation = 'sigmoid')
    self.dropout = layers.Dropout(rate = dropout_rate)

    if nb_classes == 2:
      self.dense_2 = layers.Dense(units = 1,activation = 'sigmoid')
    else:
      self.dense_2 = layers.Dense(units = nb_classes,activation = 'softmax')

  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.bigram(x_1)
    x_2 = self.pool(x_2)
    x_3 = self.bigram(x_2)
    x_3 = self.pool(x_3)

    merged = tf.concat([x_1,x_2,x_3],axis = -1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged , training)
    output = self.dense_2(merged)  
    
    return output

# 5.Training

In [29]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [30]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x) # batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1) # (batch_size, nb_filters)
        x_2 = self.trigram(x) # batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [31]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim = EMB_DIM,
            nb_filters = NB_FILTERS,
            FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)

In [32]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [33]:
checkpoint_path = "/content/drive/MyDrive/BERT INTUITION/"
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
37346/37346 [==============================] - 3843s 102ms/step - loss: 0.4213 - accuracy: 0.8062
Checkpoint saved at /content/drive/MyDrive/BERT INTUITION/.
Epoch 2/5
37346/37346 [==============================] - 3852s 103ms/step - loss: 0.3588 - accuracy: 0.8435
Checkpoint saved at /content/drive/MyDrive/BERT INTUITION/.
Epoch 3/5
37346/37346 [==============================] - 3891s 104ms/step - loss: 0.3222 - accuracy: 0.8625
Checkpoint saved at /content/drive/MyDrive/BERT INTUITION/.
Epoch 4/5
 8806/37346 [======>.......................] - ETA: 45:17 - loss: 0.2824 - accuracy: 0.8835

In [39]:
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [40]:
results = Dcnn.evaluate(test_dataset)
print(results)

4149/4149 [==============================] - 23s 6ms/step - loss: 0.3637 - accuracy: 0.8417
[0.3637111783027649, 0.8416937589645386]


In [41]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Ouput of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Ouput of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [42]:
get_prediction("this movie was awesome!")

Ouput of the model: [[0.984916]]
Predicted sentiment: positive.


# Trying Some RealWorld Negative Prediction Example

In [43]:
get_prediction('Zapier is sooooo confusing to me')

Ouput of the model: [[0.10199497]]
Predicted sentiment: negative.


In [44]:
get_prediction("Awful experience. I would never buy this product again")

Ouput of the model: [[0.1760469]]
Predicted sentiment: negative.


In [46]:
get_prediction('Your customer service is a nightmare! Totally useless!!')

Ouput of the model: [[0.00427677]]
Predicted sentiment: negative.
